In [1]:
from gensim import corpora, models
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.stem.snowball import EnglishStemmer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

from src.preprocessing import *
from src.feature_reduction import *
from src.classifiers import *
from src.plots import *
from src.CV import *

import time
from sklearn.svm import SVC
import pandas as pd
import matplotlib as mpl
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
import sklearn.preprocessing as prepro
%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
import tensorflow as tf

In [3]:
random_state=0

In [4]:
pd.set_option('display.max_colwidth',-1)
df = pd.read_json(path_or_buf='amazon_step23.json',orient='records',lines=True)

In [5]:
df.shape

(64706, 9)

In [6]:
df = df.sample(10000,random_state=random_state)
df.head()

asin  helpful  overall  \
18091  B000002NBY  [0, 0]   5         
47316  B0002ZDW80  [3, 3]   4         
44199  B0000DD79B  [2, 87]  1         
44398  B0000DFZZA  [9, 9]   4         
18616  B000002O6A  [0, 2]   4         

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

## Pre-processing

In [7]:
a = np.array([[1, 2], [3, 4]])
print (a.shape)

(2, 2)


In [8]:
# create a corpus class with an iterator that reads one corpus document per line without loading all into memory
from gensim import corpora
from nltk.stem import WordNetLemmatizer
import enchant

eng_dic = enchant.Dict("en_US")
tester = 1
lemmatizer = WordNetLemmatizer()
#Changed the stopwords from the list of step1.
stopword_file=open('stopwords_list_step2.txt')
STOPWORDS=stopword_file.read().split()

negativeword_file=open('negative_words.txt')
NEGATIVEWORDS=negativeword_file.read().split()

positiveword_file=open('positive_words.txt')
POSITIVEWORDS=positiveword_file.read().split()

negationword_file=open('negation_words.txt')
NEGATIONWORDS=negationword_file.read().split()

#Merge both reviewText and summary
documents = df[['reviewText', 'summary']].apply(lambda x: ''.join(x), axis=1)

#print ('original: ',documents[47316], '\n')

In [9]:
#Remove special characters
documents_no_specials=remove_specials_characters(documents)
# remove stop words and tokenize
documents_no_stop= []
for document in documents_no_specials:
    new_text=[]
    for word in document.lower().split():
        if word not in STOPWORDS:
            new_text.append(word)
    documents_no_stop.append(new_text)

In [10]:
#NOT SURE IF WE NEED TO REMOVE NUMERICALS
documents_no_stop_no_numeric = remove_numerical(documents_no_stop)
#print ('remove numerics: ',documents_no_stop_no_numeric[tester], '\n')

In [11]:
# lemmattizing tokens (better than stemming by taking word context into account)
documents_no_stop_no_numeric_lemmatize = [[lemmatizer.lemmatize(token) for token in text] 
                                                    for text in documents_no_stop_no_numeric]

#print ('lemmatize: ',documents_no_stop_no_numeric_lemmatize[tester], '\n')

In [12]:
# remove non-english words
documents_no_stop_no_numeric_lemmatize_english = [[token for token in text if (eng_dic.check(token)) ] 
                                                            for text in documents_no_stop_no_numeric_lemmatize]

#print ('no english: ',documents_no_stop_no_numeric_lemmatize_english[tester], '\n')

In [13]:
df['reviewCleaned'] = [" ".join(doc) for doc in documents_no_stop_no_numeric_lemmatize_english] 

#print (df['reviewCleaned'],"\n")

In [14]:
document_neg_pos = []
for row in df['reviewCleaned']:
    review = []
    new_word = ""
    for word in row.split(" "):
        if word in NEGATIONWORDS:
            new_word = word
        if word in POSITIVEWORDS:
            if not new_word:
                review.append(word)
            else:
                new_word = new_word + word
                review.append(new_word)
                new_word = ""
        if word in NEGATIVEWORDS:
            if not new_word:
                review.append(word)
            else:
                new_word = new_word + word
                review.append(new_word)
                new_word = ""
    document_neg_pos.append(review)


In [15]:
# create ready corpus
df["reviewWorded"] = [" ".join(doc) for doc in document_neg_pos]
df = df[df['reviewWorded'].map(len) > 1]
#print (df["reviewWorded"])

In [16]:
overall = df['overall'].values.tolist()

# compute values of OVERALL_BIN
OVERALL_BIN = [int(overall_value >= 5) for overall_value in overall]

# add OVERALL_BIN column to dataframe
df['OVERALL_BIN'] = OVERALL_BIN

## Vectorizing

In [17]:
data = []
print (len(df['overall']))
for element in df['overall']:
    if element == 1:
        data.append([1,0,0,0,0])
    elif element == 2:
        data.append([0,1,0,0,0])
    elif element == 3:
        data.append([0,0,1,0,0])
    elif element == 4:
        data.append([0,0,0,1,0])
    else:
        data.append([0,0,0,0,1])

#print (data)
target = np.asarray(data)
print (target.shape)

9961
(9961, 5)


In [18]:
vectorizer = CountVectorizer(max_df=0.4,min_df=2)

# fit vectorizer, carry out vectorization and display results
vectorizer.fit(df["reviewCleaned"])
documents_vec = vectorizer.transform(df["reviewCleaned"])
#print(documents_vec) # sparse matrix representation
y = df['overall']

In [19]:
X=documents_vec.toarray()
categories=np.unique(y)

In [20]:
# Scaling the data
X_scaled=[]
for doc in X:
    min_max_scaler = preprocessing.MinMaxScaler()
    X_scaled.append(np.ravel(min_max_scaler.fit_transform(doc.reshape(-1, 1))))

print (len(X_scaled[0]))
vec_size = len(X_scaled[0])

/Users/fayezmourad/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


14669


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, target, test_size=0.2, random_state=random_state)

In [22]:
X_train=np.array(X_train)
X_test=np.array(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)
print ("X train shape : ", X_train.shape)
print ("Y train shape : ",y_train.shape)
print ("X test shape : ", X_test.shape)
print ("Y test shape : ",y_test.shape)
boundary = X_train.shape[0]-100

X train shape :  (7968, 14669)
Y train shape :  (7968, 5)
X test shape :  (1993, 14669)
Y test shape :  (1993, 5)


In [23]:
print (y_test)

[[0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 1 0]
 ..., 
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 1 0 0]]


In [24]:
# encoding: UTF-8
# Copyright 2016 Google.com
import tensorflow as tf
print("Tensorflow version " + tf.__version__)
tf.set_random_seed(0)

# neural network with 1 layer of 10 softmax neurons

# input X: 1xvec_size vector
X_tensor = tf.placeholder(tf.float32, [None, vec_size])
# correct answers will go here
Y_ = tf.placeholder(tf.float32, [None, 5])
# weights W[11119, 5]
W = tf.Variable(tf.zeros([vec_size, 5]))
# biases b[5]
b = tf.Variable(tf.zeros([5]))


L = 200
M = 100
N = 60
O = 30
# Weights initialised with small random values between -0.2 and +0.2
# When using RELUs, make sure biases are initialised with small *positive* values for example 0.1 = tf.ones([K])/10
W1 = tf.Variable(tf.truncated_normal([vec_size, L], stddev=0.1))  # 784 = 28 * 28
B1 = tf.Variable(tf.zeros([L]))
W2 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
B2 = tf.Variable(tf.zeros([M]))
W3 = tf.Variable(tf.truncated_normal([M, N], stddev=0.1))
B3 = tf.Variable(tf.zeros([N]))
W4 = tf.Variable(tf.truncated_normal([N, O], stddev=0.1))
B4 = tf.Variable(tf.zeros([O]))
W5 = tf.Variable(tf.truncated_normal([O, 5], stddev=0.1))
B5 = tf.Variable(tf.zeros([5]))


Y1 = tf.nn.sigmoid(tf.matmul(X_tensor, W1) + B1)
Y2 = tf.nn.sigmoid(tf.matmul(Y1, W2) + B2)
Y3 = tf.nn.sigmoid(tf.matmul(Y2, W3) + B3)
Y4 = tf.nn.sigmoid(tf.matmul(Y3, W4) + B4)
Ylogits = tf.matmul(Y4, W5) + B5
Y = tf.nn.softmax(Ylogits)

#Define loss function
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_)
cross_entropy = tf.reduce_mean(cross_entropy)*100

# accuracy of the trained model, between 0 (worst) and 1 (best)
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# training, learning rate = 0.005
learning_rate = 0.005
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

# matplotlib visualisation
allweights = tf.reshape(W, [-1])
allbiases = tf.reshape(b, [-1])

# init
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

max_test_accuracy = 0

#for i in range(0,boundary, 100):
batch_X = X_train
batch_Y = y_train
train_data = {X_tensor: batch_X, Y_: batch_Y}
print (batch_Y.shape)
# train
sess.run(train_step, feed_dict=train_data)

# success ?
a,c = sess.run([accuracy, cross_entropy], feed_dict=train_data)
print ("Train accuracy : ",a)
print ("Train cross entropy: ",c)

test_data={X_tensor: X_test, Y_: y_test}
a,c = sess.run([accuracy, cross_entropy], feed_dict=test_data)
max_test_accuracy = max(a,max_test_accuracy)
print ("Test accuracy : ",a)
print ("Test cross entropy: ",c)

    
    
print ("\n\n\n Max test accuracy : ", max_test_accuracy)

Tensorflow version 1.1.0
(7968, 5)
Train accuracy :  0.543926
Train cross entropy:  156.72
Test accuracy :  0.533869
Test cross entropy:  156.977



 Max test accuracy :  0.533869
